In [ ]:
"""
The following env works
 - torch: 1.9.1
 - torchvision: 0.10.0+cu102
 - torch_sparse: 0.6.12
 
"""

In [ ]:
import os
PATH = '/home/chaoqiy2/github/PyHealth'
os.chdir(PATH)

In [ ]:
from pyhealth.sampler import NeighborSampler
from pyhealth.models import Graph_TorchvisionModel
from pyhealth.models import GCN
from torchvision import transforms
from pyhealth.datasets import COVID19CXRDataset

## Load Dataset

In [ ]:
from pyhealth.datasets import COVID19CXRDataset

root = "/srv/local/data/COVID-19_Radiography_Dataset"
base_dataset = COVID19CXRDataset(root)

In [ ]:
base_dataset.default_task

In [ ]:
sample_dataset = base_dataset.set_task()

In [ ]:
from torchvision import transforms


transform = transforms.Compose([
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1)),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5862785803043838], std=[0.27950088968644304])
])


def encode(sample):
    sample["path"] = transform(sample["path"])
    return sample


sample_dataset.set_transform(encode)

In [ ]:
from pyhealth.datasets import split_by_sample

# Get Index of train, valid, test set
train_index, val_index, test_index = split_by_sample(
    dataset=sample_dataset,
    ratios=[0.7, 0.1, 0.2],
    get_index = True
)

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

model = Graph_TorchvisionModel(
        dataset=sample_dataset,
        feature_keys=["path"],
        label_key="label",
        mode="multiclass",
        model_name="resnet18",
        model_config={},
        gnn_config={"input_dim": 256, "hidden_dim": 128},
    )

In [ ]:
# Build graph
# Set random = True will build random graph data
graph = model.build_graph(sample_dataset, random = True)

In [ ]:
# Define Sampler as Dataloader
train_dataloader = NeighborSampler(sample_dataset, graph["edge_index"], node_idx=train_index, sizes=[15, 10], batch_size=64, shuffle=True, num_workers=12)

# We sample all edges connected to target node for validation and test (Sizes = [-1, -1])
valid_dataloader = NeighborSampler(sample_dataset, graph["edge_index"], node_idx=val_index, sizes=[-1, -1], batch_size=64, shuffle=False, num_workers=12)
test_dataloader = NeighborSampler(sample_dataset, graph["edge_index"], node_idx=test_index, sizes=[-1, -1], batch_size=64, shuffle=False, num_workers=12)

In [ ]:
from pyhealth.trainer import Trainer

resnet_trainer = Trainer(model=model, device="cpu")
resnet_trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=valid_dataloader,
    epochs=1,
    monitor="accuracy",
)

In [ ]:
print(resnet_trainer.evaluate(test_dataloader))